# MLEM2 Algorithm

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/trip_data.csv')

In [3]:
df = df.drop(['Case'],axis=1)

In [4]:
df

,Wind,Humidity,Temperature,Trip
0,4,low,medium,yes
1,8,low,low,yes
2,4,medium,medium,yes
3,8,medium,high,maybe
4,12,low,medium,maybe
5,16,high,low,no
6,30,high,high,no
7,12,high,high,no


#### -------------------Finding numeric column---------------------

In [5]:
numeric_col = df.dtypes[df.dtypes != "object"].index[0]

In [6]:
numeric_col

'Wind'

In [7]:
sort_col = df['Wind'].sort_values()

In [8]:
df['sort_col'] = sort_col.values

In [9]:
df

,Wind,Humidity,Temperature,Trip,sort_col
0,4,low,medium,yes,4
1,8,low,low,yes,4
2,4,medium,medium,yes,8
3,8,medium,high,maybe,8
4,12,low,medium,maybe,12
5,16,high,low,no,12
6,30,high,high,no,16
7,12,high,high,no,30
